In [3]:
import os
import time
import shutil
import pathlib
import itertools
from PIL import Image
import random

# import data handling tools
import cv2
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
# import Deep learning Libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout, BatchNormalization
from tensorflow.keras import regularizers

# Ignore Warnings
import warnings
warnings.filterwarnings("ignore")

print ('modules loaded')

ModuleNotFoundError: No module named 'pandas'

In [2]:
import cv2
import os

# Path to the folder containing images
image_folder = "/Users/kisanetgabreselassie/Downloads/archive.zip"


# Load a few images
image_files = os.listdir(image_folder)[:5]  # Load the first 5 images
for image_file in image_files:
    image_path = os.path.join(image_folder, image_file)
    image = cv2.imread(image_path)

    # Check dimensions and channels
    if image is not None:
        print(f"Image: {image_file}")
        print(f"Dimensions: {image.shape[:2]} (Height x Width)")
        print(f"Channels: {image.shape[2] if len(image.shape) == 3 else 1}")
        cv2.imshow("Image", image)
        cv2.waitKey(0)
    else:
        print(f"Could not load image {image_file}")

cv2.destroyAllWindows()


NotADirectoryError: [Errno 20] Not a directory: '/Users/kisanetgabreselassie/Downloads/archive.zip'